In [2]:
import pandas as pd

df = pd.read_csv(r'D:\Data science\Python\all_kindle_review.csv')

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [4]:
df = df[['reviewText','rating']]

In [5]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [6]:
df['rating'] = df['rating'].apply(lambda x : 0 if x<3 else 1)

In [7]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [9]:
df.rating.value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [10]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Arjun
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))


In [12]:
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))


In [13]:
from bs4 import BeautifulSoup
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))


In [14]:
from bs4 import BeautifulSoup
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())


In [15]:
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [16]:
df.head()

,reviewText,rating
0,Jace Rankin may short hes nothing mess man hau...,1
1,Great short read I didnt want put I read one s...,1
2,Ill start saying first four books I wasnt expe...,1
3,Aggie Angela Lansbury carries pocketbooks inst...,1
4,I expect type book library pleased find price ...,1


In [17]:
import nltk
from nltk.tokenize import word_tokenize

# Tokenize all review texts
df['tokens'] = df['reviewText'].apply(word_tokenize)


In [18]:
from gensim.models import Word2Vec

# Train model on tokenized text
w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100)


In [19]:
import numpy as np

def get_avg_vector(tokens, model, vector_size):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(vectors, axis=0)

# Create features from avg word vectors
X = np.array([get_avg_vector(tokens, w2v_model, 100) for tokens in df['tokens']])


In [20]:
X

array([[-0.1126961 ,  0.14561477,  0.11854242, ..., -0.22555217,
        -0.00560982,  0.13577366],
       [-0.22545233,  0.1013469 , -0.1111047 , ..., -0.3468025 ,
         0.12266001,  0.1723364 ],
       [-0.20358689,  0.15349625,  0.06564482, ..., -0.3221109 ,
        -0.01837171,  0.13928917],
       ...,
       [-0.09684543,  0.1415358 , -0.05115954, ..., -0.3602267 ,
         0.13268413,  0.11566547],
       [-0.17779742,  0.02746512,  0.07378381, ..., -0.1124227 ,
        -0.01626854,  0.44398567],
       [-0.08559543,  0.1123604 , -0.08904906, ..., -0.3905861 ,
         0.03805197,  0.16448353]], dtype=float32)

In [21]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,df['rating'],test_size=0.2)

In [22]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB().fit(X_train,y_train)


In [23]:
y_pred = model.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score, classification_report

print('accuracy score', accuracy_score(y_test,y_pred))
print('classification report', classification_report(y_test,y_pred))

accuracy score 0.6783333333333333
classification report               precision    recall  f1-score   support

           0       0.49      0.68      0.57       760
           1       0.82      0.68      0.74      1640

    accuracy                           0.68      2400
   macro avg       0.66      0.68      0.66      2400
weighted avg       0.72      0.68      0.69      2400

